# M1.8 - Utilisation des Observations Terrestres de la NASA

*Partie de:* [**Données Climatiques Ouvertes**](https://github.com/OpenClimateScience/M1-Open-Climate-Data) | **Leçon Précédente** | **Leçon Suivante**

**Contenu :**

- [Organisation de notre système de fichiers](#Organisation-de-notre-système-de-fichiers)
- [Utilisation des observations climatiques de la NASA](#Utilisation-des-observations-climatiques-de-la-NASA)
  - [Téléchargement des données d'humidité du sol de niveau 3 de SMAP](#Téléchargement-des-données-d'humidité-du-sol-de-niveau-3-de-SMAP)
  - [Personnalisation d'un téléchargement via Earthdata Search](#Personnalisation-d'un-téléchargement-via-Earthdata-Search)
- [Comprendre les fichiers de données hiérarchiques (HDF5)](#Comprendre-les-fichiers-de-données-hiérarchiques-(HDF5))
  - [Lecture des ensembles de données HDF5](#Lecture-des-ensembles-de-données-HDF5)
  - [Extraction des données SMAP L3](#Extraction-des-données-SMAP-L3)
- [Masquage des plans d'eau permanents](#Masquage-des-plans-d'eau-permanents)
- [Utilisation des indicateurs d'assurance qualité](#Utilisation-des-indicateurs-d'assurance-qualité)
- [Création d'une série chronologique d'humidité du sol](#Création-d'une-série-chronologique-d'humidité-du-sol)
  - [Calcul d'une moyenne mobile](#Calcul-d'une-moyenne-mobile)

---

In [ ]:
import earthaccess
import numpy as np
import xarray as xr
from matplotlib import pyplot

auth = earthaccess.login()

## Organisation de notre système de fichiers

Encore une fois, nous aurons besoin d'un endroit pour stocker ces données brutes. Lorsque nous avons commencé à travailler avec les données de Noah NLDAS, nous avons créé les dossiers suivants dans notre système de fichiers :

```
data_raw/
  NLDAS
  SMAP_L3
```

Assurez-vous qu'il y ait également un dossier `SMAP_L3` pour recevoir les données que nous sommes sur le point de télécharger !

---

## Utilisation des observations climatiques de la NASA

Les données NLDAS que nous avons utilisées sont un excellent outil pour des études rétrospectives mais, en tant que données de réanalyse, elles présentent certaines limitations :

- Elles ont une latence relativement élevée ; il peut s'écouler des jours ou des semaines avant que les données ne soient disponibles.
- Elles intègrent des données provenant de multiples sources mais avec des niveaux d'exactitude et une couverture géographique variables.

Si nous voulons caractériser la sécheresse éclair ou la détecter en quasi temps réel, nous ne devons pas utiliser des ensembles de données de réanalyse. Au lieu de cela, nous voulons des observations directes des conditions de sécheresse. **Voyons ce que nous pouvons apprendre sur la sécheresse éclair de 2017 à partir des estimations d'humidité du sol par satellite de la NASA.**

**Nous allons utiliser les données de la mission Soil Moisture Active Passive (SMAP) de la NASA.** [Les missions d'observation de la Terre de la NASA fournissent des données regroupées en différents niveaux de traitement :](https://www.earthdata.nasa.gov/engage/open-data-services-and-software/data-information-policy/data-levels)

- **Niveau 1 (Données brutes) :** Il s'agit essentiellement des valeurs mesurées directement par un instrument satellitaire. Elles peuvent ou non être interprétables physiquement. La plupart des utilisateurs finaux ne bénéficieront pas des données de niveau 1.
- **Niveau 2 :** Ce sont des valeurs physiquement interprétables dérivées des données brutes, avec la même résolution spatiale et temporelle que les données de niveau 1. Les données de niveau 2 peuvent être difficiles à utiliser car la structure spatiale des données correspond à la géométrie de vision de l'instrument.
- **Niveau 3 :** Au niveau 3, les valeurs géophysiques ont été standardisées sur une grille spatiale uniforme et une série chronologique uniforme. Bien que certaines valeurs puissent être manquantes en raison de la faible qualité, des nuages ou de la défaillance du capteur, les données de niveau 3 en grille à différentes étapes temporelles peuvent être facilement combinées et comparées.
- **Niveau 4 (Données améliorées par modèle) :** Au niveau 4, les valeurs des données de niveau 3 sont intégrées dans un modèle, combinant éventuellement des ensembles de données supplémentaires et indépendants provenant d'autres capteurs pour produire des estimations ou des analyses améliorées des variables géophysiques.

### Téléchargement des données d'humidité du sol de niveau 3 de SMAP

**[Nous utiliserons les données d'humidité du sol de surface de niveau 3 à 36 km de la mission SMAP](https://nsidc.org/data/spl3smp/versions/8)** car elles représentent un bon compromis entre les observations directes des capteurs et la facilité d'utilisation.

- Sur le site web ci-dessus, nous pouvons voir qu'il existe plusieurs façons d'accéder aux données. [Utilisons Earthdata Search ;](https://search.earthdata.nasa.gov/search?q=SPL3SMP+V008) pouvons-nous accéder aux données depuis le cloud de la NASA en utilisant `earthaccess` ?
- Vous avez peut-être remarqué que les données de niveau 3 de SMAP que nous voulons utiliser *ne sont pas* « Disponibles dans le cloud Earthdata ». Il semble que nous devions télécharger les données directement.
- **Où allons-nous mettre les données brutes que nous téléchargeons ?** Revisitons notre arbre de fichiers dans Jupyter Notebook.
- **Dans le dossier `data_raw`, créons un nouveau dossier appelé `SMAP_L3`.** C'est ici que nous placerons les données que nous allons télécharger.

Nous avons discuté de l'importance d'avoir un flux de travail bien documenté qui permet de comprendre facilement comment nous avons obtenu un résultat scientifique particulier. Nous supposons que nous pouvons re-télécharger les données brutes que nous avons utilisées à tout moment, mais que se passerait-il si nous oublions d'où viennent les données ? Étant donné que les données de niveau 3 de SMAP ne sont pas disponibles dans le cloud, nous allons les télécharger manuellement et il serait bon de documenter les étapes que nous avons suivies pour le faire, au cas où des questions se poseraient sur l'origine des données ou le type de traitement qui leur a été appliqué.

#### &#x1F3AF; Bonne Pratique

- Dans l'arborescence du fichier Jupyter Notebook, dans le dossier `SMAP_L3`, créons un `"Nouveau Fichier"`. Nommez le nouveau fichier texte `README.txt`.
- Double-cliquez sur `README.txt` pour l'ouvrir. C'est ici que nous ajouterons des informations utiles sur les données que nous allons télécharger. Voici un exemple.

```
Auteur : Jane Q. Public (jane.public@example.com)
Date : 1er novembre 2023

Ce dossier contient les données de niveau 3 de la mission SMAP. Elles ont été téléchargées depuis :

    https://search.earthdata.nasa.gov/search?q=SPL3SMP+V008

Voici quelques informations supplémentaires sur ce produit :

    https://nsidc.org/data/spl3smp/versions/8
```

Cela peut ne pas sembler beaucoup d'informations, mais il y a suffisamment ici que nous voudrions savoir si nous faisions une longue pause dans ce projet ou si quelqu'un d'autre devait essayer de comprendre ce que nous faisions. Et sa courte longueur est également un avantage : **documenter votre projet ne doit pas être difficile et toute quantité d'informations est meilleure que rien.**

### Personnalisation d'un téléchargement via Earthdata Search

Le satellite SMAP effectue deux passages chaque jour, un passage "matinal" et un passage "après-midi" (heure locale). Utilisons les données d'humidité du sol du passage après-midi (PM), car c'est probablement à ce moment que le stress hydrique sur la végétation est à son maximum.

- [**Ce lien vous mènera à la bonne page pour commencer.**](https://search.earthdata.nasa.gov/search?q=SPL3SMP%20V008) Cliquez sur l'ensemble de données qui est affiché à droite de la fenêtre de recherche.
- Nous téléchargerons les données d'août et septembre pour étudier l'apparition et la progression de la sécheresse éclair de 2017 : **Choisissez une plage temporelle, du 1er juin 2017 au 30 septembre 2017.**
- En bas à droite, **cliquez sur le gros bouton vert qui indique "Download All" (Tout Télécharger).**
- 3,8 Go, c'est beaucoup de données ! Pouvons-nous rendre ce téléchargement plus petit ? Nous ne sommes intéressés que par l'humidité du sol du passage après-midi. **Cliquez sur "Edit Options" et, sous "Select a data access method", sélectionnez l'option "Customize".**

![](assets/M1_Earthdata_Search_SMAP-L3_customize_order.png)

- **Faites défiler vers le bas jusqu'à "Configure data customization options" et jusqu'à "Band subsetting."**
- **Dans la zone de texte qui lit "Filter", tapez `soil_moisture_dca_pm`.** Cela filtrera les variables disponibles ("bands") pour n'afficher que cette variable spécifique, qui est l'estimation de l'humidité du sol basée sur l'algorithme à double canal (DCA) pour le passage après-midi (PM).
- Pour s'assurer que nous téléchargeons uniquement les champs que nous voulons, **vous devrez décocher la case à côté de `SPL3SMP` puis recocher la case à côté de `soil_moisture_dca_pm` (voir capture d'écran ci-dessous) et chaque variable que nous voulons conserver.**

![](assets/M1_Earthdata_Search_SMAP-L3_customize_order_variables.png)

**Nous voulons télécharger les champs suivants (uniquement) :**

- `soil_moisture_dca_pm`
- `static_water_body_fraction_pm`
- `retrieval_qual_flag_dca_pm`
  
Cliquez sur "Done" en bas de ce formulaire puis sur le gros bouton vert "Download Data" (Télécharger les Données) !

#### &#x1F6A9; <span style="color:red">Attendez !</red>

Parce que nous avons sélectionné un sous-ensemble de variables, nous devrons attendre de recevoir un e-mail nous indiquant que la commande est prête. **Vous n'avez pas besoin de faire ces étapes vous-même, car j'ai déjà préparé tous les granules de données qui seraient téléchargés de cette manière.** Ils peuvent être téléchargés directement ici :

- [SMAP_L3_SPL3SMP_V008_20170601_20170930.zip](http://files.ntsg.umt.edu/data/ScienceCore/SMAP_L3_SPL3SMP_V008_20170601_20170930.zip) (Extrayez le contenu de ce fichier ZIP dans votre dossier `data_raw/SMAP_L3`)

--- 

## Comprendre les fichiers de données hiérarchiques (HDF5)

Les données SMAP de niveau 3 que nous avons téléchargées sont stockées dans un fichier de données hiérarchiques, version 5 (HDF5).

In [ ]:
import h5py

filename = 'data_raw/SMAP_L3/SMAP_L3_SM_P_20170901_R18290_001.h5'
hdf = h5py.File(filename, 'r')
hdf

Un fichier HDF5 ressemble beaucoup à un fichier netCDF4 : ce sont tous deux des fichiers hiérarchiques capables de stocker plusieurs ensembles de données et métadonnées divers dans un même fichier. Que voulons-nous dire par "hiérarchique" ? Eh bien, un fichier HDF5 ou netCDF4 est comme un arbre de fichiers, où les *ensembles de données* peuvent être organisés en différents *groupes* imbriqués, comme illustré ci-dessous. Les métadonnées, sous forme d'*attributs*, peuvent être attachées à n'importe quel ensemble de données ou groupe dans tout le fichier.

![](assets/hdf5-structure.jpg)

*Image fournie par NEON Science.*

Nous pouvons consulter les groupes et les ensembles de données au plus haut niveau de cette hiérarchie en tapant :

In [ ]:
hdf.keys()

L'objet `h5py.File`, `hdf`, est consulté comme un dictionnaire Python. Si nous voulons examiner le groupe `'Metadata'`, par exemple, nous tapons :

In [ ]:
hdf['Metadata']

Cela n'est pas très informatif, mais chaque groupe et ensemble de données dans un objet `h5py.File` se comporte également comme un dictionnaire Python :

In [ ]:
hdf['Metadata'].keys()

Le groupe `'Metadata'` est un exemple de la manière dont nous pourrions stocker des informations dans un fichier HDF5 en dehors des tableaux multi-dimensionnels.

Quelle est la signification de ce groupe vide ?

In [ ]:
hdf['Metadata/ProcessStep']

Tout comme les fichiers netCDF, chaque ensemble de données dans un fichier HDF5 peut être étiqueté avec des attributs.

In [ ]:
hdf['Metadata/ProcessStep'].attrs.keys()

In [ ]:
hdf['Metadata/ProcessStep'].attrs['processor']

### Lecture des ensembles de données HDF5

Nous ouvrons un fichier HDF5 pour lecture avec le drapeau `'r'`, ci-dessous.

In [ ]:
hdf = h5py.File(filename, 'r')
hdf.keys()

Encore une fois, nous pouvons accéder aux ensembles de données de manière hiérarchique...

In [ ]:
hdf['Soil_Moisture_Retrieval_Data_PM'].keys()

Et si nous voulons lire un ensemble de données, nous utilisons la notation `[:]` de NumPy pour indiquer que nous voulons accéder à un tableau.

In [ ]:
hdf['Soil_Moisture_Retrieval_Data_PM/soil_moisture_dca_pm'][:]

Chaque fois que nous avons fini de travailler avec un fichier HDF5 ouvert, nous devons nous assurer de le fermer.

In [ ]:
hdf.close()

**Voyons ce qui est différent lorsque nous ouvrons le même fichier en utilisant `xarray`.** En particulier, examinez les **variables de données.**

In [ ]:
ds = xr.open_dataset(filename)
ds

La seule variable trouvée, `"crs"`, ne sera pas très utile pour nous.

**`xarray` a des limitations lorsqu'il ouvre des fichiers HDF5 ; il n'est pas capable de déterminer quels groupes sont disponibles.** Au lieu de cela, nous devons spécifier le groupe que nous voulons ouvrir.

In [ ]:
ds = xr.open_dataset(filename, group = 'Soil_Moisture_Retrieval_Data_PM')
ds

Nous avons maintenant une variable utile, `"soil_moisture_dca_pm"`, mais notre `xarray.Dataset` n'a pas de coordonnées !

Une manière de résoudre cela serait d'assigner des coordonnées à notre `xarray.Dataset`. C'est pourquoi nous avons besoin de la bibliothèque `h5py`, spécialisée dans la gestion des fichiers HDF5. Nous pouvons lire les coordonnées `"x"` et `"y"` à partir de notre `h5py.File` et les ajouter au `xarray.Dataset`, comme ci-dessous.

In [ ]:
# Instructeur : Notez l'assignation des coordonnées

hdf = h5py.File(filename, 'r')
ds = ds.assign_coords({'x': hdf['x'][:], 'y': hdf['y'][:]})
hdf.close()
ds

Maintenant que nous avons à la fois une **variable de données** et des **coordonnées**, nous sommes prêts à tracer les données !

In [ ]:
pyplot.figure(figsize = (12, 5))
ds['soil_moisture_dca_pm'].plot()

#### &#x1F6A9; <span style="color:red">Faites attention</red>

**Remarquez les bandes dans cette image.** Le satellite SMAP a un temps de revisite de 2 à 3 jours. Cela signifie que, sur une seule journée, le radiomètre du satellite n'imagine qu'une partie du globe. Nous pourrions combiner les passages matinaux, `"soil_moisture_dca_am"`, et après-midi, `"soil_moisture_dca_pm"`, pour une seule journée, mais l'humidité du sol dans de nombreuses régions du monde varie beaucoup entre le matin et l'après-midi, donc cela pourrait ne pas être raisonnable.

Nous avons choisi le passage après-midi, `"soil_moisture_dca_pm"`, car c'est généralement l'après-midi que le stress hydrique est le plus élevé dans les écosystèmes terrestres.

In [ ]:
def process_smap_l3(file_path):
    '''
    Paramètres
    ----------
    file_path : str
        Le chemin vers le fichier SMAP L3

    Renvoie
    -------
    xarray.Dataset
    '''
    with h5py.File(file_path, 'r') as hdf:
        ds = xr.open_dataset(file_path, group = 'Soil_Moisture_Retrieval_Data_PM')
        return ds.assign_coords({'x': hdf['x'][:], 'y': hdf['y'][:]})

---

### Extraction des données SMAP L3

Les données d'humidité du sol de SMAP sont globales, mais nous nous intéressons actuellement à une petite région d'étude, les plaines du nord des États-Unis. Comment pouvons-nous extraire les données SMAP pour notre zone d'intérêt ?

Vous avez peut-être remarqué que les coordonnées que nous avons ajoutées à notre `xarray.Dataset`, ci-dessus, n'étaient pas des coordonnées latitude-longitude. [Les données SMAP sont projetées sur une grille EASE-Grid 2.0, où "EASE" signifie Equal-Area Scalable Earth.](https://nsidc.org/data/user-resources/help-center/guide-ease-grids#anchor-9-km-resolution-ease-grids) Cette projection unique, mondiale, présente de nombreux avantages, mais les coordonnées X ("Easting") et Y ("Northing") peuvent être difficiles à comprendre lorsque nous avons l'habitude de travailler avec des coordonnées latitude-longitude.

**Ci-dessous est un exemple de la façon dont nous pourrions modifier notre fonction `process_smap_l3()` pour convertir les coordonnées de notre ensemble de données des coordonnées EASE-Grid 2.0 en coordonnées longitude et latitude à l'aide de la bibliothèque `pyproj`.**

```python
from pyproj import Transformer
from pyproj import CRS

def process_smap_l3(file_path):
    '''
    Paramètres
    ----------
    file_path : str
        Le chemin vers le fichier SMAP L3

    Renvoie
    -------
    xarray.Dataset
    '''
    # Obtenir une fonction pour convertir les coordonnées de l'EASE-Grid 2.0 en WGS84 (latitude-longitude)
    crs_ease2 = CRS.from_epsg(6933)
    crs_wgs84 = CRS.from_epsg(4326)
    transform = Transformer.from_crs(crs_ease2, crs_wgs84)
    
    with h5py.File(file_path, 'r') as hdf:
        # Convertir de (Northing, Easting) en (Latitude, Longitude)
        lng = list(map(lambda x: transform.transform(x, 0)[1], hdf['x'][:].tolist()))
        lat = list(map(lambda y: transform.transform(0, y)[0], hdf['y'][:].tolist()))
        ds = xr.open_dataset(file_path, group = 'Soil_Moisture_Retrieval_Data_PM')
        
    return ds.assign_coords({'x': lng, 'y': lat})
```

&#x1F449; **Pour simplifier, nous ne ferons pas cette conversion des coordonnées dans la leçon d'aujourd'hui.** 

**Nous allons plutôt extraire nos données en utilisant une plage de lignes et de colonnes correspondant à une zone près de Glasgow, dans le Montana : 105.882 W de longitude et 48.059 N de latitude.**

In [ ]:
ds = process_smap_l3('data_raw/SMAP_L3/SMAP_L3_SM_P_20170802_R18290_001.h5')

ds['soil_moisture_dca_pm'][49:59,187:227].plot()

#### &#x1F6A9; <span style="color:red">Faites attention</red>

Il y a deux points à noter sur cette image :

- **Il est évident que la partie ouest de notre zone d'étude a été manquée par le satellite ce jour-là.** Cela signifie que, selon le jour, la valeur reflète les conditions d'humidité du sol dans différentes parties plus petites de notre région d'étude.
- **Il y a une zone avec une très forte humidité du sol (pixel jaune vif) au centre gauche de l'image.** Ce pixel est presque certainement un plan d'eau permanent, donc si nous faisions une analyse régionale, nous voudrions le masquer.

Abordons ces deux problèmes.

---

## Masquage des plans d'eau permanents